In [21]:
from pycaret.classification import *
from selenium import webdriver
import pandas as pd
import numpy as np
from selenium.webdriver.common.by import By
import random
from datetime import datetime, timedelta
import pickle
from fuzzywuzzy import fuzz, process
from fractions import Fraction
import time
import collections
import concurrent.futures
from collections import defaultdict
import json
import requests
from bs4 import BeautifulSoup
import statistics
import math

In [2]:
open_file = open('model_columns.pkl', "rb")
model_columns = list(pickle.load(open_file))
open_file.close()

In [3]:
scraped_cols = ['Total shots','Expected goals (xG)','Offsides','Accurate passes percentage','score','Corners','Fouls committed','Shots on target','Blocks']
pred_cols = dict(zip(model_columns, scraped_cols))

In [4]:
pred_cols

{'Shot Attempts': 'Total shots',
 'Total xG': 'Expected goals (xG)',
 'Offsides': 'Offsides',
 'Pass Completion': 'Accurate passes percentage',
 'Goals': 'score',
 'Corners': 'Corners',
 'Fouls Committed': 'Fouls committed',
 'Total Shots on Target': 'Shots on target',
 'Shots Blocked': 'Blocks'}

In [5]:
model = load_model('lr_model')

Transformation Pipeline and Model Successfully Loaded


In [22]:
today = f"{datetime.now():%Y%m%d}"
# today = '20220830'
print(today)
game_data = fotmob_game_data(f'https://www.fotmob.com/?date={today}')
all_odds = get_all_odds_links()
days_data = connect_links(game_data, all_odds)

20220830


In [23]:
days_data

{'2022-08-30 13:23:00': [['sassuolo-vs-milan',
   'ITALY - SERIE A',
   'https://www.fotmob.com/match/3919117/matchfacts/sassuolo-vs-milan',
   'https://www.oddschecker.com/football/italy/serie-a/sassuolo-v-ac-milan/winner']],
 '2022-08-30 15:23:00': [['crystal-palace-vs-brentford',
   'ENGLAND - PREMIER LEAGUE',
   'https://www.fotmob.com/match/3900978/matchfacts/crystal-palace-vs-brentford',
   'https://www.oddschecker.com/football/english/premier-league/crystal-palace-v-brentford/winner'],
  ['fulham-vs-brighton-&-hove-albion',
   'ENGLAND - PREMIER LEAGUE',
   'https://www.fotmob.com/match/3900974/matchfacts/fulham-vs-brighton-&-hove-albion',
   'https://www.oddschecker.com/football/english/premier-league/fulham-v-brighton/winner']],
 '2022-08-30 15:37:00': [['southampton-vs-chelsea',
   'ENGLAND - PREMIER LEAGUE',
   'https://www.fotmob.com/match/3900979/matchfacts/southampton-vs-chelsea',
   'https://www.oddschecker.com/football/english/premier-league/southampton-v-chelsea/winner

In [284]:
days_data = {'2022-08-29 13:53:00': [['br%C3%B8ndby-if-vs-midtjylland',
   'DENMARK - SUPERLIGAEN',
   'https://www.fotmob.com/match/3887411/matchfacts/br%C3%B8ndby-if-vs-midtjylland',
   'https://www.oddschecker.com/football/denmark/superligaen/brondby-v-midtjylland/winner']],
 '2022-08-29 14:53:00': [['cadiz-vs-athletic-club',
   'SPAIN - LALIGA',
   'https://www.fotmob.com/match/3917954/matchfacts/cadiz-vs-athletic-club',
   'https://www.oddschecker.com/football/spain/la-liga-primera/cadiz-v-athletic-bilbao/winner'],
  ['vitoria-de-guimaraes-vs-casa-pia',
   'PORTUGAL - LIGA PORTUGAL',
   'https://www.fotmob.com/match/3937388/matchfacts/vitoria-de-guimaraes-vs-casa-pia',
   'https://www.oddschecker.com/football/portugal/primeira-liga/guimaraes-v-casa-pia/winner']],
 '2022-08-29 16:53:00': [['valencia-vs-atletico-madrid',
   'SPAIN - LALIGA',
   'https://www.fotmob.com/match/3917955/matchfacts/valencia-vs-atletico-madrid',
   'https://www.oddschecker.com/football/spain/la-liga-primera/valencia-v-atletico-madrid/winner']],
 '2022-08-29 17:08:00': [['vizela-vs-gil-vicente',
   'PORTUGAL - LIGA PORTUGAL',
   'https://www.fotmob.com/match/3937386/matchfacts/vizela-vs-gil-vicente',
   'https://www.oddschecker.com/football/portugal/primeira-liga/vizela-v-gil-vicente/winner']]}

In [25]:
c = days_data.copy()
todays_data = []

while True:
    for clock in c.copy().keys():
        if datetime.strptime(clock, "%Y-%m-%d %H:%M:%S") <= datetime.now():
            stats_urls = [x[2] for x in c[clock]]
            odds_urls = [x[3] for x in c[clock]]
            print(stats_urls, odds_urls)
            with concurrent.futures.ThreadPoolExecutor() as executor:
                r = executor.map(stats_odds, stats_urls, odds_urls)
            for result in r:
                todays_data.append(result)
            del c[clock]
    print(f'{len(c)} halftimes left -- {len(todays_data)}\n')
    if len(c) == 0:
        print('all done')
        break
    time.sleep(60)

open_file = open(f'{today}_data.pkl', "wb")
pickle.dump(todays_data, open_file)
open_file.close()

7 halftimes left -- 0

7 halftimes left -- 0

7 halftimes left -- 0

7 halftimes left -- 0

7 halftimes left -- 0

7 halftimes left -- 0

7 halftimes left -- 0

7 halftimes left -- 0

7 halftimes left -- 0

7 halftimes left -- 0

7 halftimes left -- 0

7 halftimes left -- 0

7 halftimes left -- 0

7 halftimes left -- 0

7 halftimes left -- 0

7 halftimes left -- 0

7 halftimes left -- 0

7 halftimes left -- 0

7 halftimes left -- 0

7 halftimes left -- 0

7 halftimes left -- 0

7 halftimes left -- 0

7 halftimes left -- 0

7 halftimes left -- 0

7 halftimes left -- 0

7 halftimes left -- 0

7 halftimes left -- 0

7 halftimes left -- 0

7 halftimes left -- 0

7 halftimes left -- 0

7 halftimes left -- 0

7 halftimes left -- 0

7 halftimes left -- 0

7 halftimes left -- 0

7 halftimes left -- 0

7 halftimes left -- 0

7 halftimes left -- 0

7 halftimes left -- 0

7 halftimes left -- 0

7 halftimes left -- 0

7 halftimes left -- 0

7 halftimes left -- 0

7 halftimes left -- 0

7 halftimes

['Draw', 'Burnley', 'Millwall']
[2.25, 2.230769230769231, 5.5]
burnley-vs-millwall
Predicted winner is 1

draw: predicted prob: 14.000000000000002% - oddschecker: 44.44% (2.25) ---- bet no value
home: predicted prob: 83.0% - oddschecker: 44.82% (2.231) ---- bet 0.34595
away: predicted prob: 3.0% - oddschecker: 18.18% (5.5) ---- bet no value


['Draw', 'Sheffield United', 'Reading']
[6.0, 1.1666666666666667, 23.0]
sheffield-united-vs-reading
Predicted winner is 1

draw: predicted prob: 20.200000000000003% - oddschecker: 16.67% (6.0) ---- bet 0.0212
home: predicted prob: 75.2% - oddschecker: 85.69% (1.167) ---- bet no value
away: predicted prob: 4.6% - oddschecker: 4.35% (23.0) ---- bet 0.0013


['Draw', 'Cardiff', 'Luton']
[2.25, 3.0, 3.3]
cardiff-city-vs-luton-town
Predicted winner is 1

draw: predicted prob: 17.599999999999998% - oddschecker: 44.44% (2.25) ---- bet no value
home: predicted prob: 76.0% - oddschecker: 33.33% (3.0) ---- bet 0.32
away: predicted prob: 6.4% - oddschecker: 

In [278]:
open_file = open('20220828_data.pkl', "rb")
xcxc = pickle.load(open_file)
open_file.close()

In [7]:
def predicted_probs(results):
    d = {}
    for k, v in results.items():
        try:
            d[k] = float(v[0]) - float(v[1])
        except:
            d[k] = v
    for_pred = [[d[key] for key in pred_cols.values()]]
    pred = model[-1].predict_proba(for_pred)
    return pred

In [8]:
def make_bet(decimal_odds, predicted_prob, k_weight):
    if 1/decimal_odds >= predicted_prob:
        return 'no value'
    else:
        return kelly(predicted_prob, decimal_odds, k_weight)

In [9]:
def print_advice(results):
    try:
        d_h_a = ['draw'] + results['odds_url'].split('/')[-2].split('-v-')
        order_decimal_odds = [getname(odds_name, list(results['odds'].keys())) for odds_name in d_h_a]
        print(order_decimal_odds)
        p_probs = predicted_probs(results)[0]
        predicted = np.argmax(p_probs)
        decimal_odds = [results['odds'][key] for key in order_decimal_odds]
        print(decimal_odds)
        bet_made = {}
        print('\033[1m' + results['match_name'] + '\033[0m')
        print(f'Predicted winner is {predicted}\n')
        for p_prob, d_odd, win in zip(p_probs, decimal_odds, ['draw', 'home','away']):
            p_prob = round(p_prob, 3)
            d_odd = round(d_odd, 3)
            size = make_bet(d_odd, p_prob, 0.5)
            print(f'{win}: predicted prob: {p_prob*100}% - oddschecker: {round(1/d_odd*100,2)}% ({d_odd}) ---- bet {size}')
            bet_made[win] = size
        print('\n')
        bet_made['odds_checker_sum'] = np.sum([1/o for o in decimal_odds])
        bet_made['winner'] = predicted
        bet_made['predicted_probs'] = np.round(p_probs, 3)
        bets = {'bet':bet_made} 
        row = {**results, **bets}
    except:
        row = results
    return row

In [10]:
def kelly(p, b, weight):
    stake = round((p * (b - 1) - (1 - p)) / (b - 1), 4) * weight
    return stake if stake > 0.0 else 'no value'

In [11]:
def getname(name, l):
    if len(l) == 0:
        return ''
    highest = process.extractOne(name,l)
    return highest[0]

In [12]:
def get_stats(url):
    driver = webdriver.Chrome()
    try:
        driver.implicitly_wait(1)
        driver.get(url)
        ht = driver.find_elements(By.XPATH, '//span[@class="css-8ngw4i-bottomRow"]')[1].text
        if ht != 'Half-Time':
            print('not halftime just yet ... sleeping\n')
            time.sleep(3*60)
            ht = driver.find_elements(By.XPATH, '//span[@class="css-8ngw4i-bottomRow"]')[1].text
        button = driver.find_elements(By.XPATH, '(//button[@class="css-12l9xvx-ExpandButton e7i6a0l0"])[2]')[0]
        button.click()
        div1 = driver.find_elements(By.XPATH, '//li[@class="css-radwzz-Stat e683amr6"]')
        t = {k[1]: [k[0], k[2]] for k in [i.split('\n') for i in [x for x in [d.text for d in div1]
              if x not in ['Shots','Passes','Expected goals (xG)','Discipline','Defence','Duels']]]}
        score = driver.find_element(By.XPATH, '//span[@class="css-bw7eig-topRow"]').get_attribute('textContent').split(' ')
        t['score'] = [score[0], score[2]]
        posession = driver.find_element(By.XPATH, '//div[@class="css-7s52se-PossessionWheel e683amr3"]').text.split('\n')
        t['posession'] = [posession[1].replace('%',''), posession[2].replace('%','')]
        game_stats = {}
        for name, stats in t.items():
            if '(' in ''.join(stats):
                s = stats[0].split(' ') + stats[1].split(' ')
                if len(s) == 4:
                    s = [i.replace('(', '').replace(')','').replace('%','') for i in s]
                    game_stats[name] = [s[0], s[2]]
                    game_stats[name + ' percentage'] = [int(s[1])/100, int(s[3])/100]
                else:
                    s = [i for i in s if '%' not in i]
                    game_stats[name] = s
            else:
                game_stats[name] = stats
#         game_stats = {k: float(v[0]) - float(v[1]) for k, v in game_stats.items()}
        match_name = url.split('/')[-1]
        game_stats['match_name'] = match_name
        game_stats['half_time'] = ht
        game_stats['url'] = url
    except:
        game_stats = {}
    driver.quit()
    return game_stats

In [13]:
xg_leagues = {'UNITED STATES - MAJOR LEAGUE SOCCER':'world/usa/mls','ENGLAND - CHAMPIONSHIP':'english/championship',
              'ENGLAND - PREMIER LEAGUE': 'english/premier-league', 'GERMANY - BUNDESLIGA': 'germany/bundesliga',
              'FRANCE - LIGUE 1': 'france/ligue-1','AUSTRIA - BUNDESLIGA': 'austria/bundesliga',
              'PORTUGAL - LIGA PORTUGAL':'portugal/primeira-liga', 'BELGIUM - FIRST DIVISION A':'belgium/jupiler-pro-league',
              'SWITZERLAND - SUPER LEAGUE':'switzerland/super-league','SPAIN - LALIGA':'spain/la-liga-primera',
             'DENMARK - SUPERLIGAEN':'denmark/superligaen', 'MEXICO - LIGA MX APERTURA':'world/mexico/liga-mx',
             'NETHERLANDS - EREDIVISIE':'netherlands/eredivisie', 'SCOTLAND - PREMIERSHIP': 'scottish/premiership',
             'ITALY - SERIE A':'italy/serie-a'}

def fotmob_game_data(url):
    driver = webdriver.Chrome()
    driver.implicitly_wait(3)
    driver.get(url)
    divs = driver.find_elements(By.XPATH, '//div[@class="css-aoqfcq-Group e7pc1841"]')
    today = url[-8:]
    league = {}
    for count, bucket in enumerate(divs):
        league_name = bucket.find_elements(By.XPATH, '//a[@class="css-vfisw6-GroupTitleLink e7pc1842"]')[count].text
        if league_name not in xg_leagues.keys():
            continue
        url = bucket.find_elements(By.TAG_NAME, 'a')
        urls = [u.get_attribute('href') for u in url]
        time = bucket.text
        time = [datetime.strptime(i + today, '%H:%M%Y%m%d') + timedelta(minutes=53) for i in time.split('\n') if ':' in i]
        urls = urls[1:]
        names = [u.split('/')[-1] for u in urls]
        league[league_name] = {n: [u,t] for u, t, n in zip(urls, time, names)}
    return league

In [14]:
def get_all_odds_links():
    odds_links = {}
    for f_league, url_league in xg_leagues.items():
        driver = webdriver.Chrome()
        driver.implicitly_wait(3)
        url = 'https://www.oddschecker.com/football/' + url_league
        driver.get(url)
        driver.implicitly_wait(3)
        divs = driver.find_elements(By.XPATH, '//a[@class="beta-callout full-height-link whole-row-link"]')
        mt = []
        for div in divs:
            odds_url = div.get_attribute('href')
            mt.append(odds_url)
        odds_links[f_league] = mt
    return odds_links

In [15]:
def connect_links(data, all_odds):
    for l in data.keys():
        fotmob_names = data[l].keys()
        odds_names = {v: k for k, v in zip(all_odds[l],[str(i).split('/')[-2] for i in all_odds[l]])}
        for game in fotmob_names:
            match = getname(game, odds_names.keys())
            try:
                link = odds_names[match]
            except:
                link = ''
            data[l][game] = data[l][game] + [link]
    scraper_data = collections.defaultdict(list)
    for l_name, data in data.items():
        for game_name, l1_t_l2 in data.items():
            scraper_data[str(l1_t_l2[1])].append([game_name, l_name, l1_t_l2[0], l1_t_l2[2]])
    r = {}
    test = scraper_data.copy()
    for date, data in test.copy().items():
        if len(data) > 5:
            x = [data[i:i + 4] for i in range(0, len(data), 4)]
            d = datetime.strptime(date, "%Y-%m-%d %H:%M:%S")
            groups = len(x)
            times = [str(d + timedelta(minutes = i * 1)) for i in range(-groups//2+1,groups//2+1, 1)]
            new = dict(zip(times, x))
            del test[date]
            r = {**r, **new}
        else:
            r[date] = data
    scraper_data = dict(sorted(r.items(), key=lambda x: x[0]))
    return scraper_data

In [16]:
def get_odds(url):
    try:
        driver = webdriver.Chrome()
        if url == '':
            return {'odds_url':url, 'odds': '', 'full_odds': ''}
        driver.implicitly_wait(1)
        driver.get(url)
        html = driver.page_source
        soup = BeautifulSoup(html, features="html.parser")
        odds_containers = soup.find_all("div", {"class": "oddsAreaWrapper_o17xb9rs RowLayout_refg9ta"})
        names = soup.find_all("div", {"class": "BetRowLeftBetContent_b2f00kt"})
        names = [n.text for n in names]
        odds_containers = [container.find_all('button') for container in odds_containers]
        odds = [[z.text for z in container] for container in odds_containers]
        odds = [[float(Fraction(j.replace(' ',''))) + 1.0 for j in o][:-3] for o in odds]
        median_odds = [statistics.median(o) for o in odds]
        median_odds = dict(zip(names, median_odds))
        odds = {'odds_url':url, 'odds': median_odds, 'full_odds': odds}
    except:
        return {'odds_url':url, 'odds': '', 'full_odds': ''}
    driver.quit()
    return odds

In [17]:
def stats_odds(stats_url, odds_url):
    stats = get_stats(stats_url)
    time.sleep(20)
    odds = get_odds(odds_url)
    row = {**stats, **odds}
    result = print_advice(row)
    return result